<a href="https://colab.research.google.com/github/PhillipMas/PhillipMas/blob/main/HYPER_PARAMETER_TUNING_RRN_WINDOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install keras-tuner -q


In [12]:

import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import keras
import keras_tuner as kt

In [ ]:


# Fetch data
ticker = '^J203.JO'  # Example ticker
data = yf.download(ticker, start='2012-01-01', end='2024-07-21')
data['Log Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
data = data.dropna()  # Drop NaN values

# Prepare data for LSTM
window_size = 120
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data['Log Returns'].values.reshape(-1, 1))

# Function to create rolling windows
def create_rolling_windows(data, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

# Create rolling windows
X, y = create_rolling_windows(scaled_data, window_size)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Define model building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(LSTM(units=hp.Int(f'units_{i}', min_value=50, max_value=200, step=50), return_sequences=True if i < hp.Int('num_layers', 1, 3) - 1 else False, input_shape=(X.shape[1], 1)))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(1))
    # Use keras.optimizers instead of kt.optimizers
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), loss='mean_squared_error')
    return model


# Initialize Keras Tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='my_dir',
    project_name='LSTM_tuning'
)

# Split data into training and validation sets
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# Perform hyperparameter tuning
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
model = build_model(best_hps)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Predict returns for each rolling window
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)

# Calculate VaR for each rolling window
VaR_estimates = []
for i in range(window_size, len(data)):
    window_predictions = predictions[i-window_size:i]
    VaR = np.percentile(window_predictions, 5)  # 5% VaR
    VaR_estimates.append(VaR)


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
150               |150               |units_0
0                 |0                 |dropout_0
0.00015999        |0.00015999        |learning_rate

Epoch 1/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 18s 205ms/step - loss: 1.0202 - val_loss: 1.0922
Epoch 2/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 15s 204ms/step - loss: 0.9476 - val_loss: 1.0911
Epoch 3/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 21s 214ms/step - loss: 0.9497 - val_loss: 1.0914
Epoch 4/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 20s 206ms/step - loss: 0.9138 - val_loss: 1.0914
Epoch 5/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 21s 212ms/step - loss: 0.9903 - val_loss: 1.0919
Epoch 6/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 20s 202ms/step - loss: 0.9115 - val_loss: 1.0902
Epoch 7/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 21s 204ms/step - loss: 1.0272 - val_loss: 1.0928
Epoch 8/50
76/76 ━━━━━━━━━━━━━━━━━━━━ 16s 217ms/step - loss: 1.0069 - val_loss: 1.0921
Epoch 9/50
76/76 ━━━━━━━━━━━